In [36]:
from ortools.sat.python import cp_model

#crear CSP
model = cp_model.CpModel()
#variables
#variable de paises de cada equipo
pais = [model.NewIntVar(1,10,'pais_'+str(i)) for i in range(32)]
#cantidad = [model.NewIntVar(1,100,'cant_'+str(i)) for i in range(32)]

#variable g_i_j que significa que el equipo i está en el grupo j
grupos = []
for i in range(32):
  grupos += [[model.NewBoolVar('grupo_'+str(i)+'_'+str(j)) for j in range(8)]]
#print(grupos)

#restricciones
#Arg  Bra Ecu Par Col Per Uru Chi Bol Ven
#6    7   3   2   4   2   2   2   2   2

#distribuir paises
for i in range(32):
  if i < 6:
    model.Add(pais[i] == 1)
  elif i < 13:
    model.Add(pais[i] == 2)
  elif i < 16:
    model.Add(pais[i] == 3)
  elif i < 18:
    model.Add(pais[i] == 4)
  elif i < 22:
    model.Add(pais[i] == 5)
  elif i < 24:
    model.Add(pais[i] == 6)
  elif i < 26:
    model.Add(pais[i] == 7)
  elif i < 28:
    model.Add(pais[i] == 8)
  elif i < 30:
    model.Add(pais[i] == 9)
  else:
    model.Add(pais[i] == 10)

#cada equipo está en un solo grupo
for i in range(32):
  #print("->",grupos[i])
  #model.Add(sum([grupos[i][j] for j in range(8)]) == 1)
  model.Add(sum(grupos[i]) == 1)

#cada grupo tiene 4 equipos
for j in range(8):
  model.Add(sum([grupos[i][j] for i in range(32)]) == 4)

#cada grupo tiene 4 equipos de distintos paises
p_tmp1 = []#para multiplicar pais por presencia en grupo, así tener 0 o el valor del pais
p_tmp2 = []
for i in range(32):
  p_tmp1 += [[model.NewIntVar(0,100,'g_'+str(i)+'_'+str(j)) for j in range(8)]]
  p_tmp2 += [[model.NewIntVar(0,100,'g_'+str(i)+'_'+str(j)) for j in range(8)]]
for i in range(32):
  for j in range(8):
    model.AddMultiplicationEquality(p_tmp1[i][j],[pais[i],grupos[i][j]])
    model.Add(p_tmp2[i][j] == p_tmp1[i][j]).OnlyEnforceIf(grupos[i][j])

#para cada grupo todos los valores distintos, lo que significa que son de paises distintos
for j in range(8):
  model.AddAllDifferent([p_tmp2[i][j] for i in range(32)])

In [37]:
solver = cp_model.CpSolver()
status = solver.Solve(model)
if status == cp_model.OPTIMAL:
  #header
  print('\t',end='')
  for i in range(32):
    print(solver.Value(pais[i]),end='\t')
  print()
  print('\t',end='')
  for i in range(32):
    print(i+1,end='\t')
  print()
  ################
  for j in range(8):
    print('G'+str(j+1),end='->\t')
    for i in range(32):
      print(solver.Value(grupos[i][j]),end='\t')
    print()
  
  print()
  ################
  for j in range(8):
    print('G'+str(j+1),end='->\t')
    for i in range(32):
      print(solver.Value(p_tmp1[i][j]),end='\t')
    print()
  
  print()
  ################
  for j in range(8):
    print('G'+str(j+1),end='->\t')
    for i in range(32):
      print(solver.Value(p_tmp2[i][j]),end='\t')
    print() 
  
  print()
  nombre_pais = ['Arg', 'Bra', 'Ecu', 'Par', 'Col','Per','Uru','Chi', 'Bol', 'Ven']
  nombre_equipo = ['River Plate','Boca Juniors','Defensa y Justicia','Racing Club','Vélez Sarsfield','Argentinos Juniors',
                   'Palmeiras','Flamengo','São Paulo','Internacional','Atlético Mineiro','Fluminense','Santos',
                   'Liga de Quito','Barcelona','Independiente del Valle',
                   'Cerro Porteño','Olimpia',
                   'Santa Fe','América de Cali','Atlético Nacional','Junior',
                   'Sporting Cristal','Universitario',
                   'Nacional','Rentistas',
                   'Universidad Católica','Unión La Calera',
                   'The Strongest','Always Ready',
                   'Deportivo Táchira','Deportivo La Guaira']
  for j in range(8):
    print('Grupo ',(j+1),end='->\t')
    for i in range(32):
      if solver.Value(grupos[i][j]):
        print(nombre_equipo[i][:10]+'('+nombre_pais[solver.Value(pais[i]) - 1]+')',end='\t')
    print()

	1	1	1	1	1	1	2	2	2	2	2	2	2	3	3	3	4	4	5	5	5	5	6	6	7	7	8	8	9	9	10	10	
	1	2	3	4	5	6	7	8	9	10	11	12	13	14	15	16	17	18	19	20	21	22	23	24	25	26	27	28	29	30	31	32	
G1->	0	0	1	0	0	0	0	1	0	0	0	0	0	0	0	1	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	
G2->	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	1	1	0	0	0	0	0	0	0	
G3->	1	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	0	1	0	0	0	
G4->	0	0	0	0	1	0	0	0	0	0	0	0	1	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	1	
G5->	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	1	0	0	0	1	0	
G6->	0	0	0	1	0	0	0	0	0	1	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	
G7->	0	0	0	0	0	1	0	0	0	0	0	1	0	1	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	
G8->	0	0	0	0	0	0	0	0	1	0	0	0	0	0	0	0	0	0	0	0	0	1	0	0	0	1	0	1	0	0	0	0	

G1->	0	0	1	0	0	0	0	2	0	0	0	0	0	0	0	3	4	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	
G2->	0	1	0	0	0	0	0	0	0	0	0	0	0	0	0	0	0	4	0	0	0	0	0	6	7	0	0	0	0	0	0	0	
G3->	1	0	0	0	0	0	0	0	0	0	2	0	0	0	0	0	0	0	0	0	5	0	0	0	0	0	0	0	9	0	0	0	
G4->	0	0	0	0	1	0	0	0	0	0	0	0	2	0	0	0	0	0	5	0	0	0	0	0	0	0	0	0	0	0	0	10	
G